In [1]:
import pandas as pd
import numpy as np

### Load Original Datasets

In [2]:
df1 = pd.read_csv(r"asserts\Electric_Vehicle_Population_Data.csv")
df1.head(2)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5UXTA6C03P,King,Seattle,WA,98177.0,2023,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,30,0,36.0,218985539,POINT (-122.38242499999996 47.77279000000004),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
1,1FMCU0EZXN,Yakima,Moxee,WA,98936.0,2022,FORD,ESCAPE,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,15.0,197264322,POINT (-120.37951169999997 46.55609000000004),PACIFICORP,5.307700e+10


In [3]:
df2 = pd.read_csv(r"asserts\Electric_Vehicle_Population_Size_History_By_County.csv")
df2.head(2)

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,November 30 2020,St. Mary's,MD,Passenger,1,0,1,215,216,0.46
1,February 28 2021,Sedgwick,KS,Passenger,1,1,2,89,91,2.20


# Cleaning

### Filter Out Needed Columns and Rows

In [4]:
# filter out only Washington and California records
df1_wa_ca = df1[df1['State'].isin(['WA', 'CA'])].reset_index(drop=True)
df2_wa_ca = df2[df2['State'].isin(['WA', 'CA'])].reset_index(drop=True)

In [5]:
#Select relevant columns for both datasets(Based on insights I want to draw from both datasets)

df1_filtered = df1_wa_ca[['State', 'County', 'City', 'Model Year', 'Make',
                          'Model', 'Electric Vehicle Type', 'Electric Range', 'Base MSRP']].copy()



df2_filtered = df2_wa_ca[['State', 'County', 'Date', 'Electric Vehicle (EV) Total', 'Non-Electric Vehicle Total',
                          'Battery Electric Vehicles (BEVs)', 'Plug-In Hybrid Electric Vehicles (PHEVs)']].copy()


In [6]:
# Rename specific columns and rearrange columns for both datasets

df1_filtered.rename(columns={
    'Model Year': 'ModelYear',
    'Electric Vehicle Type': 'EVType',
    'Electric Range': 'ElectricRange',
    'Base MSRP': 'BasePrice'
}, inplace=True)


df1_filtered = df1_filtered[['State', 'County', 'City', 'Make', 'Model', 'ModelYear', 'EVType', 'ElectricRange', 'BasePrice']]


df2_filtered.rename(columns={
    'Electric Vehicle (EV) Total': 'EVTotal',
    'Non-Electric Vehicle Total': 'NonEVTotal',
    'Battery Electric Vehicles (BEVs)': 'BEVCount',
    'Plug-In Hybrid Electric Vehicles (PHEVs)': 'PHEVCount'
}, inplace=True)


df2_filtered = df2_filtered[['Date', 'State', 'County', 'BEVCount', 'PHEVCount', 'EVTotal', 'NonEVTotal']]



### Check Filtered Datasets Info

In [7]:
df1_filtered.head()


,State,County,City,Make,Model,ModelYear,EVType,ElectricRange,BasePrice
0,WA,King,Seattle,BMW,X5,2023,Plug-in Hybrid Electric Vehicle (PHEV),30,0
1,WA,Yakima,Moxee,FORD,ESCAPE,2022,Plug-in Hybrid Electric Vehicle (PHEV),38,0
2,WA,King,Seattle,CHEVROLET,BOLT EV,2018,Battery Electric Vehicle (BEV),238,0
3,WA,King,Newcastle,TESLA,MODEL S,2013,Battery Electric Vehicle (BEV),208,69900
4,WA,Kitsap,Bremerton,FORD,C-MAX,2015,Plug-in Hybrid Electric Vehicle (PHEV),19,0


In [8]:
df2_filtered.head()

,Date,State,County,BEVCount,PHEVCount,EVTotal,NonEVTotal
0,April 30 2021,WA,Thurston,0,0,0,56710
1,October 31 2017,WA,Adams,3,3,6,12283
2,July 31 2018,WA,Whatcom,2,0,2,42313
3,August 31 2019,WA,Grays Harbor,0,0,0,20570
4,October 31 2019,WA,Cowlitz,0,0,0,29023


In [9]:
df1_filtered.info()
print("\n\n")
df2_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143359 entries, 0 to 143358
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   State          143359 non-null  object
 1   County         143357 non-null  object
 2   City           143357 non-null  object
 3   Make           143359 non-null  object
 4   Model          143359 non-null  object
 5   ModelYear      143359 non-null  int64 
 6   EVType         143359 non-null  object
 7   ElectricRange  143359 non-null  int64 
 8   BasePrice      143359 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 9.8+ MB



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7543 entries, 0 to 7542
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        7543 non-null   object
 1   State       7543 non-null   object
 2   County      7543 non-null   object
 3   BEVCount    7543 non-null   int64 
 4   PH

### Handling Missing Values for Both Datasets

In [10]:
# Check missing values for df1_filtered
missing_values_df1 = df1_filtered.isna().sum()
print("Missing values in df1_filtered:")
print(missing_values_df1)


Missing values in df1_filtered:
State            0
County           2
City             2
Make             0
Model            0
ModelYear        0
EVType           0
ElectricRange    0
BasePrice        0
dtype: int64


In [11]:
# Check missing values for df2_filtered
missing_values_df2 = df2_filtered.isna().sum()
print("Missing values in df2_combined_insights:")
print(missing_values_df2)


Missing values in df2_combined_insights:
Date          0
State         0
County        0
BEVCount      0
PHEVCount     0
EVTotal       0
NonEVTotal    0
dtype: int64


In [12]:
# Find the rows where 'County' or 'City' is missing in df1_filtered
missing_county_or_city = df1_filtered[df1_filtered['County'].isna() | df1_filtered['City'].isna()]

print("Rows with missing County or City values:")
print(missing_county_or_city)


Rows with missing County or City values:
       State County City   Make    Model  ModelYear  \
126446    CA    NaN  NaN  TESLA  MODEL 3       2018   
136203    CA    NaN  NaN  TESLA  MODEL 3       2022   

                                EVType  ElectricRange  BasePrice  
126446  Battery Electric Vehicle (BEV)            215          0  
136203  Battery Electric Vehicle (BEV)              0          0  


In [13]:
# Replacing the missing values with a placeholder
df1_filtered.fillna({'County': 'Unknown', 'City': 'Unknown'}, inplace=True)


### Data Type Conversion

In [14]:
df2_filtered['Date'] = pd.to_datetime(df2_filtered['Date'])


# Transforming 

### Rename the Values in "EVType" Column

In [17]:
#checking the EVType value counts
print(df1_filtered["EVType"].value_counts())


df1_filtered['EVType'] = df1_filtered['EVType'].replace({
    'Battery Electric Vehicle (BEV)': 'BEV',
    'Plug-in Hybrid Electric Vehicle (PHEV)': 'PHEV'
})


BEV     110716
PHEV     32643
Name: EVType, dtype: int64


### Checking Both Datasets for Merging 

In [21]:
# Filter the datasets for each state
df1_wa = df1_filtered[df1_filtered['State'] == 'WA']
df1_ca = df1_filtered[df1_filtered['State'] == 'CA']

df2_wa = df2_filtered[df2_filtered['State'] == 'WA']
df2_ca = df2_filtered[df2_filtered['State'] == 'CA']

#Get the unique counties for each state in both datasets
unique_counties_df1_wa = df1_wa['County'].unique()
unique_counties_df1_ca = df1_ca['County'].unique()

unique_counties_df2_wa = df2_wa['County'].unique()
unique_counties_df2_ca = df2_ca['County'].unique()

In [23]:
#Compare the unique counties to see if they match:
#For WA 
common_counties_wa = set(unique_counties_df1_wa).intersection(set(unique_counties_df2_wa))
only_in_df1_wa = set(unique_counties_df1_wa) - set(unique_counties_df2_wa)
only_in_df2_wa = set(unique_counties_df2_wa) - set(unique_counties_df1_wa)

print("Common counties in WA:", common_counties_wa)
print("Counties only in df1 for WA:", only_in_df1_wa)
print("Counties only in df2 for WA:", only_in_df2_wa)


Common counties in WA: {'Skamania', 'Okanogan', 'Thurston', 'Kitsap', 'Clallam', 'Grant', 'Yakima', 'Klickitat', 'Adams', 'Douglas', 'San Juan', 'Jefferson', 'Franklin', 'Garfield', 'Benton', 'Lincoln', 'Kittitas', 'Ferry', 'Stevens', 'Asotin', 'Spokane', 'Snohomish', 'Columbia', 'Skagit', 'Whitman', 'Pacific', 'King', 'Island', 'Walla Walla', 'Wahkiakum', 'Lewis', 'Grays Harbor', 'Chelan', 'Pierce', 'Clark', 'Pend Oreille', 'Mason', 'Cowlitz', 'Whatcom'}
Counties only in df1 for WA: set()
Counties only in df2 for WA: set()


In [24]:
#For CA
common_counties_ca = set(unique_counties_df1_ca).intersection(set(unique_counties_df2_ca))
only_in_df1_ca = set(unique_counties_df1_ca) - set(unique_counties_df2_ca)
only_in_df2_ca = set(unique_counties_df2_ca) - set(unique_counties_df1_ca)

print("Common counties in CA:", common_counties_ca)
print("Counties only in df1 for CA:", only_in_df1_ca)
print("Counties only in df2 for CA:", only_in_df2_ca)


Common counties in CA: {'Kern', 'Santa Cruz', 'San Bernardino', 'Orange', 'Napa', 'Kings', 'Marin', 'Sacramento', 'Monterey', 'Sonoma', 'Ventura', 'Santa Clara', 'Alameda', 'Riverside', 'Solano', 'Contra Costa', 'Placer', 'Los Angeles', 'Santa Barbara', 'San Diego'}
Counties only in df1 for CA: {'Unknown'}
Counties only in df2 for CA: {'San Mateo', 'San Luis Obispo', 'San Francisco', 'Fresno', 'Shasta', 'Tuolumne', 'San Joaquin', 'El Dorado', 'Tulare'}


## Insight One:

Adoption Rates and Geographic Distribution 

Big question:  How is EV adoption distributed across Washington and California, both geographically and by time?

To answer the big question, below are several helper questions.

### To Answer Question One (merge both datasets):

Which county in WA has seen the most significant growth in EV adoption over a specific time period?

In [28]:
#To get the number of EVs registered in each county for each time period (ModelYear).
df1_wa_agg = df1_wa.groupby(['County', 'ModelYear']).agg({
    'Make': 'count',
}).reset_index().rename(columns={'Make': 'NumOfEVs'})


In [29]:
df1_wa_agg.head()

,County,ModelYear,NumOfEVs
0,Adams,2011,1
1,Adams,2012,2
2,Adams,2014,2
3,Adams,2015,2
4,Adams,2016,1


In [30]:
#To get  the EVTotal in each county for each date
df2_wa_agg = df2_wa.groupby(['County', 'Date']).agg({
    'EVTotal': 'sum',  
}).reset_index()



In [31]:
df2_wa_agg.head()

,County,Date,EVTotal
0,Adams,2017-01-31,2
1,Adams,2017-02-28,4
2,Adams,2017-03-31,6
3,Adams,2017-04-30,5
4,Adams,2017-05-31,5


In [32]:
#Merge df1_wa_agg and df2_wa_agg on the "County"
df_wa_merged = pd.merge(df1_wa_agg, df2_wa_agg, on='County', how='inner')


In [33]:
df_wa_merged.head()

,County,ModelYear,NumOfEVs,Date,EVTotal
0,Adams,2011,1,2017-01-31,2
1,Adams,2011,1,2017-02-28,4
2,Adams,2011,1,2017-03-31,6
3,Adams,2011,1,2017-04-30,5
4,Adams,2011,1,2017-05-31,5


### For Shen: Now this is the dataset you can analysis to answer the above question. 

we can compare specific time period here. First, we can see which county in (2017) has significant growth, then change to a different time period(2022).

### To Answer Question Two (Dataset 2):

How do different types of EV (BEV vs PHEV) fare in different counties in WA and CA?

In [36]:
#To get the number of eah type of vehicle group by county(WA) and date(yearly)

#get "year" column

df2_wa['Year'] = df2_wa['Date'].dt.year

df2_wa_yearly = df2_wa.groupby(['County', 'Year']).agg({
    'BEVCount': 'sum',
    'PHEVCount': 'sum',
    'EVTotal': 'sum',
    'NonEVTotal': 'sum'
}).reset_index()


C:\Users\qfu88\AppData\Local\Temp\ipykernel_33544\918321979.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_wa['Year'] = df2_wa['Date'].dt.year


In [37]:
df2_wa_yearly.head()

,County,Year,BEVCount,PHEVCount,EVTotal,NonEVTotal
0,Adams,2017,30,38,68,207838
1,Adams,2018,59,57,116,213190
2,Adams,2019,61,72,133,215174
3,Adams,2020,99,83,182,217697
4,Adams,2021,165,130,295,221049


In [39]:
#To get the number of eah type of vehicle group by county(CA) and date(yearly)
df2_ca['Year'] = df2_ca['Date'].dt.year

df2_ca_yearly = df2_ca.groupby(['County', 'Year']).agg({
    'BEVCount': 'sum',
    'PHEVCount': 'sum',
    'EVTotal': 'sum',
    'NonEVTotal': 'sum'
}).reset_index()


C:\Users\qfu88\AppData\Local\Temp\ipykernel_33544\2158458783.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_ca['Year'] = df2_ca['Date'].dt.year


In [40]:
df2_ca_yearly.head()

,County,Year,BEVCount,PHEVCount,EVTotal,NonEVTotal
0,Alameda,2017,17,12,29,4411
1,Alameda,2018,0,16,16,4114
2,Alameda,2019,41,19,60,3647
3,Alameda,2020,31,5,36,3230
4,Alameda,2021,42,0,42,2791


### Datasets with All States

In [44]:
df_all = df2[['State', 'County', 'Date', 'Electric Vehicle (EV) Total', 'Non-Electric Vehicle Total',
                          'Battery Electric Vehicles (BEVs)', 'Plug-In Hybrid Electric Vehicles (PHEVs)']].copy()

#Rename and Rearrange columns
df_all.rename(columns={
    'Electric Vehicle (EV) Total': 'EVTotal',
    'Non-Electric Vehicle Total': 'NonEVTotal',
    'Battery Electric Vehicles (BEVs)': 'BEVCount',
    'Plug-In Hybrid Electric Vehicles (PHEVs)': 'PHEVCount'
}, inplace=True)


df_all = df_all[['Date', 'State', 'County', 'BEVCount', 'PHEVCount', 'EVTotal', 'NonEVTotal']]

df_all['Date'] = pd.to_datetime(df_all['Date'])

df_all.dropna(inplace=True)


In [52]:
df_all['Year'] = df_all['Date'].dt.year


df_all_yearly = df_all.groupby(['Year', 'State']).agg({
    'BEVCount': 'sum',
    'PHEVCount': 'sum',
    'EVTotal': 'sum',
    'NonEVTotal': 'sum'
}).reset_index()


In [53]:
df_all_yearly.head()

,Year,State,BEVCount,PHEVCount,EVTotal,NonEVTotal
0,2017,AK,0,12,12,582
1,2017,AL,12,12,24,1194
2,2017,AZ,58,20,78,29064
3,2017,CA,264,148,412,89981
4,2017,CO,20,6,26,9623


### For Shen: use above datasets to answer question2

we can do several comparison: BEV and PHEV changing over time in the two states, EVtotal and NonEvtotal  changing over time in the two states.

Last dataset, we gonna compare all states!

### To Answer Question Three (WA Dataset):

How has the Electric Range of EVs influenced adoption (for year 2021) rates in different counties of WA?

In [59]:
# filter out data for 2021
df2_wa_2021 = df2_wa[df2_wa['Year'] == 2021]

#Aggregate Data by County

df2_wa_2021_agg = df2_wa_2021.groupby(['County']).agg({
    'BEVCount': 'sum',
    'PHEVCount': 'sum',
    'EVTotal': 'sum',
    'NonEVTotal': 'sum'
}).reset_index()





In [61]:
#merge with df1_wa based on "County"

df_wa_merged = pd.merge(df1_wa, df2_wa_2021_agg, on='County', how='inner')

In [63]:
#Group by County and Electric Range, and sum the 'EVTotal', 'BEVCount', 'PHEVCount' to analyze adoption based on range.
df_wa_merged_grouped = df_wa_merged.groupby(['County', 'ElectricRange']).agg({
    'EVTotal': 'sum',
    'BEVCount': 'sum',
    'PHEVCount': 'sum'
}).reset_index()


In [65]:
df_wa_merged_grouped.head()

,County,ElectricRange,EVTotal,BEVCount,PHEVCount
0,Adams,0,4425,2475,1950
1,Adams,16,295,165,130
2,Adams,19,590,330,260
3,Adams,21,590,330,260
4,Adams,25,295,165,130


### For Shen: use the above dataset for question3

you can use the 'EVTotal', 'BEVCount', and 'PHEVCount' to approximate the adoption rate for different electric ranges. Then you can analyze whether higher or lower electric ranges are correlated with higher adoption rates.







## Insights two:

Insight 2: Popular Types of Electric Vehicles and their Attributes
Big Question: What types of electric vehicles are most popular, and what are their typical characteristics like price range, electric range, etc.?


To answer the big question, below are several helper questions.

### To Answer Question one:

Are there regional preferences for certain types of electric vehicles? For example, are BEVs more popular in urban areas, while PHEVs are preferred in rural areas?

In [71]:
#create two datasets(urban dataset and rural dataset)

urban_cities = ['Seattle', 'Bellevue', 'Redmond', 'Vancouver', 'San Francisco', 'Palo Alto', 'San Jose']
df_urban = df1_filtered[df1_filtered['City'].isin(urban_cities)]

rural_counties = ['Kern', 'Sonoma', 'Napa', 'Monterey', 'Santa Barbara', 'Okanogan', 'Stevens', 'Wahkiakum', 'Ferry', 'Pend Oreille', 'Columbia', 'Garfield']
df_rural = df1_filtered[df1_filtered['County'].isin(rural_counties)]



### For Shen: above two datasets used for question one

I sliced out some typical cities as urban area, and counties that can consider as rural area. 

To answer above question, we can just do some exploratory data analysis and statistic tests on these two datasets.

### To Answer Question Two:

Which makes and models are the most popular in WA and CA?

In [72]:
#group data by make and model
popular_makes_models_wa = df1_wa.groupby(['Make', 'Model']).size().reset_index(name='Count').sort_values('Count', ascending=False)
popular_makes_models_ca = df1_ca.groupby(['Make', 'Model']).size().reset_index(name='Count').sort_values('Count', ascending=False)


### For Shen: above two datasets used for question one

This is easiest one, we can just use these two dataset create bar charts, then interpret the chart